In [1]:
import pandas as pd
import os
import numpy as np
import openai
import gradio as gr
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY
literary_score_df= pd.read_csv('Resources/literary_scored_superheroes.csv')
def simulate_battle(score_a, score_b):
    total_score = score_a + score_b
    prob_a_wins = score_a / total_score
    outcome = np.random.rand()  # Generates a random float between 0 and 1
    return 1 if outcome < prob_a_wins else 0  # Returns 1 if A wins, 0 if B wins
def describe_battle(hero1, hero2):
    # Correctly fetch scores for hero1 and hero2 before using them
    try:
        for index, row in literary_score_df.iterrows():
            if hero1 in row['aliases']:
                name_a=row['name']
                score_a=row['hero_total_score']
            if hero2 in row['aliases']:
                name_b=row['name']
                score_b=row['hero_total_score']
    except IndexError:
        return "One of the heroes not found in the dataset", ""
    # Simulate battle outcome based on scores
    winner = hero1 if simulate_battle(score_a, score_b) == 1 else hero2
    # Generate the battle description using GPT (simplified example)
    try:
        response = openai.Completion.create(
            model="gpt-3.5-turbo-instruct",  # Adjust based on your OpenAI API plan
            prompt=f"Describe an epic battle between {hero1} and {hero2}, where {winner} wins.",
            temperature=0.7,
            max_tokens=500
        )
        battle_description = response.choices[0].text.strip()
    except Exception as e:
        return "Failed to generate description", str(e)
    return winner, battle_description
# Interface setup
iface = gr.Interface(
    fn=describe_battle,
    inputs=[gr.Textbox(label="Hero 1"), gr.Textbox(label="Hero 2")],
    outputs=[gr.Textbox(label="Winner"), gr.Textbox(label="Battle Description")]
)
iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [4]:
import pandas as pd
import os
import numpy as np
import openai
import gradio as gr
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY

literary_score_df = pd.read_csv('Resources/literary_scored_superheroes.csv')

def simulate_battle(score_a, score_b):
    total_score = score_a + score_b
    prob_a_wins = score_a / total_score
    outcome = np.random.rand()  # Generates a random float between 0 and 1
    return 1 if outcome < prob_a_wins else 0  # Returns 1 if A wins, 0 if B wins

def describe_battle(hero1, hero2):
    # Initialize scores to None to detect if heroes are found
    score_a = None
    score_b = None
    
    # Fetch scores for hero1 and hero2
    for index, row in literary_score_df.iterrows():
        # Check each hero in aliases
        if hero1.lower() in row['aliases'].lower():
            score_a = row['hero_total_score']
        if hero2.lower() in row['aliases'].lower():
            score_b = row['hero_total_score']

    # Check if either hero was not found and return error message if so
    if score_a is None or score_b is None:
        return ("One of the heroes not found in the dataset.", "", None)

    # Simulate battle outcome based on scores
    winner = hero1 if simulate_battle(score_a, score_b) == 1 else hero2

    # Generate the battle description using GPT
    try:
        response = openai.Completion.create(
            model="gpt-3.5-turbo-instruct",
            prompt=f"Describe an epic battle between {hero1} and {hero2}, where {winner} wins.",
            temperature=0.7,
            max_tokens=500
        )
        battle_description = response.choices[0].text.strip()
    except Exception as e:
        return ("Failed to generate description", str(e), None)
    
    return (winner, battle_description, None)

# Interface setup
iface = gr.Interface(
    fn=describe_battle,
    inputs=[gr.Dropdown(label="Hero 1"), gr.Dropdown(label="Hero 2")],
    outputs=[gr.Textbox(label="Winner"), gr.Textbox(label="Battle Description")]
)

iface.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [6]:
import pandas as pd
import os
import numpy as np
import openai
import gradio as gr
from dotenv import load_dotenv
from PIL import Image
import requests
from io import BytesIO

# Load environment variables from .env file
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY

literary_score_df = pd.read_csv('Resources/literary_scored_superheroes.csv')

def simulate_battle(score_a, score_b):
    total_score = score_a + score_b
    prob_a_wins = score_a / total_score
    outcome = np.random.rand()  # Generates a random float between 0 and 1
    return 1 if outcome < prob_a_wins else 0  # Returns 1 if A wins, 0 if B wins

def describe_battle(hero1, hero2):
    # Initialize scores to None to detect if heroes are found
    score_a = None
    score_b = None
    
    # Fetch scores for hero1 and hero2
    for index, row in literary_score_df.iterrows():
        # Check each hero in aliases
        if hero1.lower() in row['aliases'].lower():
            score_a = row['hero_total_score']
        if hero2.lower() in row['aliases'].lower():
            score_b = row['hero_total_score']

    # Check if either hero was not found and return error message if so
    if score_a is None or score_b is None:
        return ("One of the heroes not found in the dataset.", "", None, None)

    # Simulate battle outcome based on scores
    winner = hero1 if simulate_battle(score_a, score_b) == 1 else hero2

    # Generate the battle description using GPT
    try:
        response = openai.Completion.create(
            model="gpt-3.5-turbo-instruct",
            prompt=f"Describe an epic battle between {hero1} and {hero2}, where {winner} wins within openai safety guidelines.",
            temperature=0.7,
            max_tokens=500
        )
        battle_description = response.choices[0].text.strip()

        # Generate an image based on the battle description
        image_response = openai.Image.create(
            model="dall-e-3",
            prompt=battle_description,
            n=1,
            size="1024x1024"
        )
        image_url = image_response['data'][0]['url']
        image_data = requests.get(image_url).content
        image = Image.open(BytesIO(image_data))
    except Exception as e:
        return ("Failed to generate description or image", str(e), None, None)
    
    return (winner, battle_description, image)
heroes=list(literary_score_df['name'])
# Interface setup
iface = gr.Interface(
    fn=describe_battle,
    inputs=[gr.Dropdown(heroes, label='Hero 1'),gr.Dropdown(heroes, label='Hero 2') ],
    outputs=[gr.Textbox(label="Winner"), gr.Textbox(label="Battle Description"), gr.Image(label="Battle Image")]
)

iface.launch()


Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\Robert\.conda\envs\dev\lib\site-packages\gradio\queueing.py", line 522, in process_events
    response = await route_utils.call_process_api(
  File "c:\Users\Robert\.conda\envs\dev\lib\site-packages\gradio\route_utils.py", line 260, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Users\Robert\.conda\envs\dev\lib\site-packages\gradio\blocks.py", line 1689, in process_api
    result = await self.call_function(
  File "c:\Users\Robert\.conda\envs\dev\lib\site-packages\gradio\blocks.py", line 1255, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\Users\Robert\.conda\envs\dev\lib\site-packages\anyio\to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *args, cancellable=cancellable,
  File "c:\Users\Robert\.conda\envs\dev\lib\site-packages\anyio\_backends\_asyncio.py", line 818, in run_sync_in_worker_thread
    return await future
  F

In [2]:
import pandas as pd
import os
import numpy as np
import openai
import gradio as gr
from dotenv import load_dotenv
from PIL import Image
import requests
from io import BytesIO

# Load environment variables from .env file
load_dotenv("open_ai_key.env")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY

literary_score_df = pd.read_csv('Resources/literary_scored_superheroes.csv')

def simulate_battle(score_a, score_b):
    total_score = score_a + score_b
    prob_a_wins = score_a / total_score
    outcome = np.random.rand()  # Generates a random float between 0 and 1
    return 1 if outcome < prob_a_wins else 0  # Returns 1 if A wins, 0 if B wins

def describe_battle(hero1, hero2):
    # Initialize scores to None to detect if heroes are found
    score_a = None
    score_b = None
    
    # Fetch scores for hero1 and hero2
    try:
        score_a = literary_score_df.loc[literary_score_df['name'].str.lower() == hero1.lower(), 'hero_total_score'].iloc[0]
        score_b = literary_score_df.loc[literary_score_df['name'].str.lower() == hero2.lower(), 'hero_total_score'].iloc[0]
    except IndexError:
        return "One of the heroes not found in the dataset", ""

    # Check if either hero was not found and return error message if so
    

    # Simulate battle outcome based on scores
    winner = hero1 if simulate_battle(score_a, score_b) == 1 else hero2

    # Generate the battle description using GPT
    try:
        response = openai.Completion.create(
            model="gpt-3.5-turbo-instruct",
            prompt=f"Describe an epic battle between {hero1} and {hero2}, where {winner} wins within openai safety guidelines.",
            temperature=0.7,
            max_tokens=500
        )
        battle_description = response.choices[0].text.strip()

        # Generate an image based on the battle description
        image_response = openai.Image.create(
            model="dall-e-3",
            prompt=battle_description,
            n=1,
            size="1024x1024"
        )
        image_url = image_response['data'][0]['url']
        image_data = requests.get(image_url).content
        image = Image.open(BytesIO(image_data))
    except Exception as e:
        return ("Failed to generate description or image", str(e), None, None)
    
    return (winner, battle_description, image)
heroes=list(literary_score_df['name'])
# Interface setup
iface = gr.Interface(
    fn=describe_battle,
    inputs=[gr.Dropdown(heroes, label='Hero 1'),gr.Dropdown(heroes, label='Hero 2') ],
    outputs=[gr.Textbox(label="Winner"), gr.Textbox(label="Battle Description"), gr.Image(label="Battle Image")]
)

iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
